In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model, metrics
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
%matplotlib inline

In [2]:
pBrownheroin = pd.read_excel('./tables/pricePurity/purityBrownHeroinMean.xlsx', 
                            nrows = 31, usecols = [i for i in range(25)]).drop('Study',
                            axis = 1).drop(29) # we gotta drop UK-E (its on row 29) 
                            # because it in the Deaths dataframe
pCannabis = pd.read_excel('./tables/pricePurity/purityCannabisMean.xlsx')
pCocaine = pd.read_excel('./tables/pricePurity/purityCocaineMean.xlsx')
pEctasy = pd.read_excel('./tables/pricePurity/purityEctasyMean.xlsx')
pWhiteheroin = pd.read_excel('./tables/pricePurity/purityWhiteHeroinMean.xlsx')

In [3]:
dMale = pd.read_excel('./tables/drugDeaths/maleDeaths.xlsx', usecols = [i for i in range(24)])
dFemale = pd.read_excel('./tables/drugDeaths/femaleDeaths.xlsx', usecols = [i for i in range(24)])
countries = dMale['Country'].replace('*','')
Deaths = dMale + dFemale
Deaths['Country'] = countries
Deaths.drop(Deaths.tail(1).index,inplace=True)

Deaths['Country'] = Deaths['Country'].str.replace('*','').str.rstrip()
pBrownheroin['Country'] =pBrownheroin['Country'].str.replace("United Kingdom-A", "United Kingdom") 

In [4]:
def get_mae_r2(suppliers, users):
    data = dict()
    i = 0
    for target_country in users['Country']:
        # making the dataframe for target_country
        df = suppliers[suppliers['Country'] == target_country].T[1:]
        df.columns = ["Purity " + target_country]

        for country in list(suppliers['Country']):
            if country != target_country:
                joiner = suppliers[suppliers['Country'] == country].T[1:]
                joiner.columns = ["Purity " + country]
                df = df.join(joiner)

        # users in target country
        joiner = users[users['Country'] == target_country].T[1:]

        if not joiner[i].isnull().sum() >= len(joiner/2):
            i+= 1
            joiner = joiner.fillna(joiner.mean())

            joiner.columns = ["Deaths " + target_country]
            df = df.join(joiner)

            # handeling NAN values
            for country in df.columns:
                if df[country].isnull().sum() >= len(df[country]/2):
                    df = df.drop(country, axis = 1) # if more than half are nan
                else:
                    df[country] = df[country].fillna(df[country].mean())

            X = df.loc[:, df.columns != "Deaths " + target_country].values
            y = df["Deaths " + target_country].values

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) 

            regressor = LinearRegression()
            #regressor = Lasso(tol=0.001) # za da ne ima errors
            #regressor = Ridge()
            #regressor = RandomForestRegressor()
            
            regressor.fit(X_train, y_train)

            y_pred = regressor.predict(X_test)

            mae=metrics.mean_absolute_error(y_test, y_pred)
            r2=r2_score(y_test, y_pred)

            data[target_country] = (mae, r2)
        else:
            i+=1
    return data

In [5]:
brownheroin = get_mae_r2(pBrownheroin, Deaths)

In [8]:
brownheroin

{'Austria': (40.17747316580403, -5.434517963343478),
 'Belgium': (54.34520460238018, -8.521405631642214),
 'Bulgaria': (19.039866877265634, -3.4125333606275223),
 'Croatia': (32.74976246185007, -3.2060198698738054),
 'Cyprus': (3.962542192830127, -2.04072515133686),
 'Czechia': (12.366418271341447, -2.444568441174669),
 'Denmark': (38.98878270540882, -25.47248844366656),
 'Estonia': (45.9113912113806, -1.9834264581851877),
 'Finland': (68.28330950829128, -4.77517386396624),
 'France': (325.34161850558206, -41.085990344588396),
 'Germany': (227.13525127449807, -4.168872011763752),
 'Greece': (157.1052175592002, -110.89909667700422),
 'Hungary': (15.747661959452225, -2.794209206646674),
 'Ireland': (101.95168668249548, -11.253363455313886),
 'Italy': (409.524198602609, -0.26432847671687476),
 'Latvia': (16.353150814503227, -40.333105413253094),
 'Lithuania': (33.60478249303172, -2.047721838714239),
 'Luxembourg': (6.8955240832148705, -0.5689755174600462),
 'Malta': (3.627468034369541, -3